<a href="https://colab.research.google.com/github/Lee-Kiwon/17th_Conference_AntiAginGAN-for-Cat/blob/main/LLM/LangChain_07_Retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install openai langchain tiktoken sentence_transformers chromadb pypdf

  Using cached openai-1.10.0-py3-none-any.whl (225 kB)
  Using cached langchain-0.1.4-py3-none-any.whl (803 kB)
  Using cached tiktoken-0.5.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.0 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 kB 8

In [5]:
import os

os.environ['OPENAI_API_KEY'] = '3a9d04966a3c43ca92de7be4b803d6f7'
os.environ['AZURE_OPENAI_ENDPOINT'] = 'https://skyfly-001.openai.azure.com/'
os.environ['OPENAI_API_TYPE'] = 'azure'
os.environ['OPENAI_API_VERSION'] = '2023-05-15'

In [6]:
#토큰 단위로 데이터 쪼개는 부분
import tiktoken

tokenizer = tiktoken.get_encoding('cl100k_base')

def tiktoken_len(text):
  tokens = tokenizer.encode(text)
  return len(tokens)

In [7]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader('/content/[정책브리프 2021-04] 탄소중립 대응을 위한 정부 정책과 동향 (1).pdf')
pages = loader.load_and_split()

In [8]:
# Data Split하는 부분
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
                  chunk_size=500,
                  chunk_overlap=50,
                  length_function=tiktoken_len
)

docs = text_splitter.split_documents(pages)

In [9]:
#Embedding model을 준비한다.
from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = 'jhgan/ko-sbert-nli'
model_kwargs = {'device':'cpu'}
encode_kwargs = {'normalize_embeddings':True}

hf = HuggingFaceBgeEmbeddings(
    model_name = model_name,
    model_kwargs = model_kwargs,
    encode_kwargs = encode_kwargs
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.46k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/538 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
#Vector store에 데이터를 저장한다.
from langchain.vectorstores import Chroma

db = Chroma.from_documents(docs,hf)

In [15]:
from langchain.chains import RetrievalQA
from langchain.chat_models import AzureChatOpenAI

openai = AzureChatOpenAI(
    deployment_name = 'dev-gpt-35-turbo'
)

qa = RetrievalQA.from_chain_type(
    llm = openai,
    chain_type='stuff',
    retriever = db.as_retriever(
        search_type = 'mmr',
        search_kwargs={'k':3, 'fetch_k':10}
    ),
    return_source_documents=True
)

In [19]:
query = '탄소 중립 사회란 무엇인가'

result = qa(query)
print(result)

{'query': '탄소 중립 사회란 무엇인가', 'result': '탄소 중립 사회란 전체적으로 탄소 배출량을 최소화하고, 남은 배출량을 탄소 흡수 등으로 상쇄하여 순배출량을 0으로 만드는 사회를 말합니다. 이는 온실가스 배출로 인한 기후 변화 문제에 대응하기 위한 글로벌 추세로서, 개인, 기업, 정부 등 모든 구성원이 탄소 중립을 실현하기 위해 노력해야 합니다. 탄소 중립 사회를 구현하기 위해서는 온실가스 배출을 줄이는 기술 개발과 에너지 효율화, 재생에너지 확대, 산업구조의 저탄소화 등 다양한 대책이 필요합니다.', 'source_documents': [Document(page_content='01이노비즈 정책브리프\n01 탄소중립과 대응 필요성\n탄소중립이란 ?\n∙탄소중립은 개인·회사·단체 등에서 배출되는 탄소와 흡수되는 탄소량을 같게 해 탄소 ‘순배출이 제\n로(0)’가 되게 하는 것으로 , ‘넷-제로(Net-Zero)’ 라 부르기도 함\n- 인간의 활동에 의한 온실가스 * 배출을 최대한 줄이고 , 남은 온실가스는 흡수, 제거하여 ‘순배출이 제로\n(0)’가 되는 개념이며 , 기후변화에 대응하기 위한 글로벌 추세로 자리 잡고 있음\n*온실가스란 ‘지구 대기를 오염시켜 온실 효과를 일으키는 가스’를 통틀어 이르는 말로, 이산화탄소 (CO2), 메탄(CH4), 아산화질소 (N2O), 수\n소불화탄소 (HFCs), 과불화탄소 (PFCs), 육불화황 (SF6) 등은 배출 규제 및 거래의 대상이 되고 있음 \n대응 필요성\n∙지구 온난화로 인한 폭염, 폭설, 태풍, 산불 등 이상기후 현상이 세계 곳곳에서 발생되고 있는 가\n운데 기후변화 문제에 대한 국제사회의 공동 대응 필요성이 대두됨', metadata={'page': 3, 'source': '/content/[정책브리프 2021-04] 탄소중립 대응을 위한 정부 정책과 동향 (1).pdf'}), Document(page_content='이노비즈 정책브리프\nI n n o b i z  P o l i c y

In [20]:
result['result']

'탄소 중립 사회란 전체적으로 탄소 배출량을 최소화하고, 남은 배출량을 탄소 흡수 등으로 상쇄하여 순배출량을 0으로 만드는 사회를 말합니다. 이는 온실가스 배출로 인한 기후 변화 문제에 대응하기 위한 글로벌 추세로서, 개인, 기업, 정부 등 모든 구성원이 탄소 중립을 실현하기 위해 노력해야 합니다. 탄소 중립 사회를 구현하기 위해서는 온실가스 배출을 줄이는 기술 개발과 에너지 효율화, 재생에너지 확대, 산업구조의 저탄소화 등 다양한 대책이 필요합니다.'